In [47]:
import torch
import torchvision
import sys
import numpy as np
import matplotlib.pyplot as plt
import os

In [48]:
from segment_anything import sam_model_registry
from segment_anything.separate_sam_encoder import SamEncoder

sam_checkpoint = "/home/icb/hanyi.zhang/main_master_thesis/segment-anything/sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

sam_image_encoder = SamEncoder(sam, device)

In [49]:
# load an image here
from PIL import Image
import numpy as np

# Function to load an image from a file and convert it to a numpy array
def load_image_to_numpy(file_path):
    # Load the image file
    with Image.open(file_path) as img:
        # Ensure the image is in RGB format
        img = img.convert('RGB')
        
        # Convert the image to a NumPy array
        image_array = np.array(img)
    
    return image_array

In [51]:
# Specify the directories for source images and target embeddings
image_folder = "/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop"
embed_folder = "/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop_emb"
os.makedirs(embed_folder)

# List all files in the image directory
images = os.listdir(image_folder)

# Iterate over each image file
i = 1
for img in images:
    if i % 50 == 0:
        print(i)

    # Construct full path to the image file
    img_path = os.path.join(image_folder, img)
    # Load the image file into a numpy array
    img_np = load_image_to_numpy(img_path)
    
    # Process the image to get its embedding
    # Assuming the 'set_image' method processes the image and returns its embedding directly
    # If 'set_image' doesn't return the embedding but updates an internal state, you'd need to access the embedding differently
    img_embed = sam_image_encoder.set_image(img_np)

    # Replace the original extension with 'npy' for the output file
    embed_path = os.path.join(embed_folder, img.replace('.png', '.npy'))
    
    # Save the embedding to the specified path
    np.save(embed_path, img_embed.cpu().detach().numpy())

    i+=1


50


In [13]:
example = np.load('/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/train/img_emb/cell_00001.npy')
example.shape

(1, 256, 64, 64)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import matplotlib.colors as mcolors

# Function to read TIFF file and convert it to a numpy array
def tiff_to_numpy(file_path):
    with Image.open(file_path) as img:
        numpy_array = np.array(img)
    return numpy_array

In [39]:
# Test one example
train_labels_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Training/labels'
train_coord_path = '/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/train/labels'
val_labels_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels'
val_coord_path = '/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/labels'

START_SYMBOL = (0, 0, 0, 0)
PADDING_SYMBOL = (-1, -1, -1, -1)
MAX_SEQ_LEN = 2000

i = 1
# Loop through each file in the directory
for file_name in os.listdir(val_labels_path):
    if file_name.endswith('.tiff'):
        file_path = os.path.join(train_labels_path, file_name)
        numpy_array = tiff_to_numpy(file_path)
        transformed_coord = sam_image_encoder.set_box_coordinates(numpy_array)

        # Add start symbol
        coords_with_start = [START_SYMBOL] + transformed_coord.tolist()

        # Add padding symbols to reach MAX_SEQ_LEN
        padding_length = MAX_SEQ_LEN - len(coords_with_start)
        if padding_length > 0:
            coords_with_start.extend([PADDING_SYMBOL] * padding_length)
        
        # Ensure the length is exactly MAX_SEQ_LEN
        coords_with_start = np.array(coords_with_start[:MAX_SEQ_LEN], dtype=np.float32)
        
        # Save the coordinates as a numpy file
        save_path = os.path.join(val_coord_path, file_name.replace('.tiff', '.npy'))
        np.save(save_path, coords_with_start)

        #print(f"Processed and saved {file_name} to {save_path}")
    
    if i % 50 == 0:
        print(i)
    i += 1



50


In [40]:
file_path = '/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/train/labels/cell_00001_label.npy'
coord = np.load(file_path)
coord

array([[  0. ,   0. ,   0. ,   0. ],
       [  0. , 272.4,  40.8, 344. ],
       [199.2, 737.2, 254.4, 767.6],
       ...,
       [ -1. ,  -1. ,  -1. ,  -1. ],
       [ -1. ,  -1. ,  -1. ,  -1. ],
       [ -1. ,  -1. ,  -1. ,  -1. ]], dtype=float32)